In [1]:
from astropy.io import fits
from scipy import *
import numpy as np
from pylab import *
from astropy.coordinates import SkyCoord
import astropy.units as units
from IPython.display import Image
from astropy.coordinates import FK5, ICRS
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import match_coordinates_sky
import WLanalysis
from scipy.interpolate import interp2d,NearestNDInterpolator
import os

In [2]:
def read_spec(fitsfile):    
    ihdulist = fits.open(fitsfile)
    idate = ihdulist[0].header['DATE-OBS'][:10]
    ispec = ihdulist[1].data
    ilambda = ihdulist[4].data
    return ilambda, ispec, idate

In [6]:
apoid_candidates = os.listdir('specs_visit')
ifitsfn = 'specs_visit/%s'%(os.listdir('specs_visit/%s'%(apoid_candidates[5]))[0])
print ifitsfn

specs_visit/apVisit-r8-6224-56561-237.fits
